In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/intents.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

121 documents
43 classes ['HR_related_problem', 'Location', 'Weather', 'about', 'appointment status', 'cabin', 'check_leave', 'commission', 'competitors_in_market', 'configuration', 'connect_people', 'cost_lowering', 'customer_satisfaction', 'domain', 'email_id', 'factors_impacting_sale', 'forgot_password', 'gadgets', 'goodbye', 'greeting', 'highest_grossing', 'hours', 'invalid', 'key_customers', 'leave', 'maintainence', 'manufacturing_problems', 'missing_id', 'name', 'noans', 'options', 'order_components', 'order_tracking', 'predict_delay', 'predict_performance', 'project_handling_queries', 'search_department', 'search_person_by_id', 'solve_problems', 'supplier_info', 'thanks', 'turnover', 'version_update']
241 unique lemmatized words ["'s", ',', '.', '23a12', '23a31', '32712', '345a23', '431b67', '561a24', '562b78', '@', 'a', 'abc', 'abx', 'accident', 'ai', 'am', 'an', 'analysis', 'and', 'anyone', 'appointment', 'appoitment', 'are', 'at', 'awesome', 'based', 'be', 'been', 'benefit', 

In [5]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('/content/chatbot_model.h5', hist)

print("model created")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
25/25 [==============================] - 14s 2ms/step - loss: 3.8192 - accuracy: 0.0032 
Epoch 2/200
25/25 [==============================] - 0s 2ms/step - loss: 3.6949 - accuracy: 0.0563
Epoch 3/200
25/25 [==============================] - 0s 1ms/step - loss: 3.6516 - accuracy: 0.1081
Epoch 4/200
25/25 [==============================] - 0s 1ms/step - loss: 3.5650 - accuracy: 0.1471
Epoch 5/200
25/25 [==============================] - 0s 1ms/step - loss: 3.5207 - accuracy: 0.1475
Epoch 6/200
25/25 [==============================] - 0s 2ms/step - loss: 3.4969 - accuracy: 0.0804
Epoch 7/200
25/25 [==============================] - 0s 1ms/step - loss: 3.3708 - accuracy: 0.1437
Epoch 8/200
25/25 [==============================] - 0s 1ms/step - loss: 3.2858 - accuracy: 0.1486
Epoch 9/200
25/25 [==============================] - 0s 1ms/step - loss: 3.1331 - accuracy: 0.1629
Epoch 10/200
25/25 [==============================] - 0s 1ms/step - loss: 2.9070 - accuracy: 0.2122
Epoch 1

In [7]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('/content/intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [8]:
chatbot_response("Hi")

'Good to see you again'